# Tutorial: Model Builder - A Data-Centric Way to Train your own Obejct Detection Models

# 1. Data Preparation

In [ ]:
!python -m pip install git+https://github.com/cqels/vision.git --force
%cd mmdetection/

In [ ]:
import shutil
data_path = 'data/image_dataset/coco/'
annotated_path = ''
if annotated_path:
    os.makedirs(annotated_path, exist_ok=True)
    if annotated_path:
    os.makedirs(annotated_path, exist_ok=True)

    for filename in os.listdir(data_path):
        source_file = os.path.join(data_path, filename)
        destination_file = os.path.join(annotated_path, filename)
    
        if os.path.isfile(source_file):
            shutil.copy(source_file, destination_file)
            print(f"Copied: {source_file} to {destination_file}")
else:
    print("Error: annotated_path is not defined.")

# 2. Object Detection in Practice starting from Model Builder

## 2.1 Prepare and set parameters for training and evaluation

In [ ]:
from os.path import join as opj
from shutil import copy
from torch_model_zoo import utils
from mmdetection_configs import configs_fcos_mb
import json

path_to_anno_mixedDatasets = '/home/mb/mmdetection/data/image_dataset/coco/annotation.json'
with open(path_to_anno_mixedDatasets) as file:
    result = json.load(file)
print(result['images'][0])
params = utils.prepare_for_training(path_to_anno_mixedDatasets, result, existed_data='/home/mb/mmdetection/data/image_dataset/coco/', filter_cat_nms=None)
path_to_config = 'configs/fcos/fcos_visionKG.py'
# with open(path_to_anno_mixedDatasets) as file:
#     config = json.load(file)
#     print(config)
path_to_work_dir = 'mixedDatasets/logs_visionKG/'
copy(configs_fcos_mb.__file__, path_to_config)
nms_categories = params['CAT_NMS']
num_categories = len(nms_categories)

## 2.2 Data-Playground

In [ ]:
if num_categories > 4:
  cat_nms = nms_categories[0:4]
else:
  cat_nms = nms_categories
utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.3 Perform Training & Evaluation on your chosen Object Detection tool

In [ ]:
# Training based on the queried MixedDataset
# For more params-setting, please check:
# https://mmdetection.readthedocs.io/en/latest/
%run tools/train.py {path_to_config} \
--cfg-options model.bbox_head.num_classes={num_categories} \
train_dataloader.dataset.metainfo.classes="$nms_categories" val_dataloader.dataset.metainfo.classes="$nms_categories"